In [1]:
with open("input.txt") as f:
    lines = [
        list(line.strip()) for line in f.readlines()
    ]

In [2]:
from itertools import product
from typing import Generator, List, Set, Tuple

Grid = List[List[str]]
Cell = Tuple[int, ...]

class ConwaySim:

    def __init__(self, initial_state: Grid, dimensions: int):
        self._active: Set[Cell] = {
            (x, y,) + (0,) * (dimensions-2)
            for y, row in enumerate(initial_state)
            for x, cell in enumerate(row)
            if cell == "#"
        }
        self._min_dimensions = (0,) * dimensions
        max_x = len(initial_state[0])
        max_y = len(initial_state)
        self._max_dimensions = (max_x, max_y) + (0, ) * (dimensions-2)
        self._num_dimensions = dimensions

    def _neighbors(self, cell: Cell) -> Generator[Cell, None, None]:
        """Returns neighboring cells from a given cell"""
        # Create a generator expression containing the ranges
        # along each dimension for the neighboring cell.
        # For example, if we want to find the neighbor of the
        # cell (0, 0, 0) then we generate these ranges
        # (range(-1, 2), range(-1, 2), range(-1, 2))
        cell_ranges = (range(coord-1, coord+2) for coord in cell)
        # Take all combinations of values across the ranges
        # from each dimension except for the cell itself to
        # get all neighboring cells.
        return (
            neighbor
            for neighbor in product(*cell_ranges)
            if neighbor != cell
        )

    def _is_active(self, cell: Cell) -> bool:
        return cell in self._active

    def _num_active_neighbors(self, cell: Cell) -> int:
        return sum(
            self._is_active(neighbor)
            for neighbor in self._neighbors(cell)
        )

    def _round(self):
        """Run a round of the simulation and update state"""
        new_active = set()
        min_dimensions = (float("inf"),) * self._num_dimensions
        max_dimensions = (float("-inf"),) * self._num_dimensions

        cell_ranges = (
            range(min_dim-1, max_dim+2)
            for min_dim, max_dim in zip(self._min_dimensions, self._max_dimensions)
        )

        for cell in product(*cell_ranges):
            if self._is_active(cell):
                if self._num_active_neighbors(cell) in {2, 3}:
                    new_active.add(cell)
            elif self._num_active_neighbors(cell) == 3:
                new_active.add(cell)

            if cell in new_active:
                min_dimensions = tuple(
                    min(first, second)
                    for first, second in zip(cell, min_dimensions)
                )
                max_dimensions = tuple(
                    max(first, second)
                    for first, second in zip(cell, max_dimensions)
                )

        self._active = new_active
        self._min_dimensions = min_dimensions
        self._max_dimensions = max_dimensions

    def num_active(self):
        return len(self._active)

    def simulate(self, num_rounds: int):
        for _ in range(num_rounds):
            self._round()

        return self.num_active()

In [3]:
ConwaySim(lines, dimensions=3).simulate(6)

317

In [4]:
ConwaySim(lines, dimensions=4).simulate(6)

1692